In [6]:
from datasets import load_from_disk

# dataset = load_from_disk("/appdataset/train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415")
# train_dataset = load_from_disk("/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_train_3.3M_0415")
# validation_dataset = load_from_disk("/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_validation_3.3M_0415")
test_dataset = load_from_disk("/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415")

# from collections import defaultdict
# from tqdm import tqdm

# task_cnt = defaultdict(int)

# for data in tqdm(test_dataset):
#     task_cnt[data['task']] += 1
    
# task_cnt

In [7]:
import pandas as pd

# Pandas Series로 변환 후 value_counts 실행
task_counts = pd.Series(test_dataset['task']).value_counts()

print(task_counts)

presto-retrosynthesis                 100
presto-forward_reaction_prediction    100
alchemy_homo                          100
aqsol-logS                            100
alchemy_homo_lumo_gap                 100
alchemy_lumo                          100
Name: count, dtype: int64


In [8]:
import random
from collections import defaultdict
from tqdm import tqdm

# 1. 각 Task별로 데이터가 어디에 있는지 인덱스(Index) 수집
# (전체 데이터를 로드하지 않고 'task' 컬럼만 읽으므로 매우 빠릅니다)
task_indices = defaultdict(list)
for idx, task in enumerate(tqdm(test_dataset['task'], desc="Grouping tasks")):
    task_indices[task].append(idx)

# 2. 각 Task별로 100개씩 랜덤 인덱스 추출
final_indices = []
seed = 42  # 재현성을 위한 시드 설정

for idx, (task, indices) in enumerate(task_indices.items()):
    # 데이터가 100개보다 적으면 있는 거 다 가져오고, 많으면 100개만 뽑음
    n_samples = min(len(indices), 100)
    sampled_indices = random.sample(indices, n_samples)
    final_indices.extend(sampled_indices)
    if idx > 4:
        break

# 3. 추출한 인덱스로 새로운 데이터셋 생성 (select는 메모리를 거의 안 씁니다)
# 마지막에 shuffle()을 해주는 것이 학습/테스트 시 편향을 막아줍니다.
temp_test_dataset = test_dataset.select(final_indices).shuffle(seed=seed)

print(f"Original Size: {len(test_dataset)}")
print(f"Sampled Size: {len(temp_test_dataset)}")

# 4. 저장하기
# 기존 경로 뒤에 '_sampled_100' 등을 붙여서 원본 덮어쓰기를 방지합니다.
original_path = "/home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415"
save_path = original_path + "_sampled_100"

print(f"Saving to: {save_path}")
temp_test_dataset.save_to_disk(save_path)

Grouping tasks: 100%|██████████| 600/600 [00:00<00:00, 790135.76it/s]


Original Size: 600
Sampled Size: 600
Saving to: /home/jovyan/CHJ/Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_sampled_100


Saving the dataset (1/1 shards): 100%|██████████| 600/600 [00:00<00:00, 21733.65 examples/s]


In [28]:
import numpy as np

np.unique(train_dataset['task'], return_counts=True)

(array(['bace', 'chebi-20-mol2text/0', 'chebi-20-text2mol/0',
        'forward_reaction_prediction', 'qm9_homo', 'qm9_homo_lumo_gap',
        'qm9_lumo', 'reagent_prediction', 'retrosynthesis',
        'smol-forward_synthesis/0', 'smol-molecule_captioning/0',
        'smol-molecule_generation/0', 'smol-name_conversion-i2s/0',
        'smol-name_conversion-s2i/0', 'smol-property_prediction-bbbp/0',
        'smol-property_prediction-clintox/0',
        'smol-property_prediction-esol/0',
        'smol-property_prediction-hiv/0',
        'smol-property_prediction-lipo/0',
        'smol-property_prediction-sider/0', 'smol-retrosynthesis/0'],
       dtype='<U34'),
 array([  1210,  25854,  25854, 121890, 117660, 117660, 117708, 121896,
        125565, 971351,  48531,  48532, 299890, 299890,   1569,   1144,
           888,  32863,   3360,  22820, 938530]))

In [ ]:
mol2text = test_dataset.filter(lambda x: x['task'] == 'chebi-20-mol2text/0')
for key, value in mol2text[0].items():
    print(key, ":",value)
    

x : [[5, 0, 4, 5, 3, 0, 2, 0, 0], [7, 0, 2, 5, 0, 0, 1, 0, 0], [5, 0, 3, 5, 0, 0, 1, 0, 0], [7, 0, 1, 5, 0, 0, 1, 0, 0], [5, 0, 3, 5, 1, 0, 1, 0, 0], [5, 0, 3, 5, 0, 0, 1, 0, 1], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 1, 4, 5, 1, 0, 2, 0, 1], [5, 2, 4, 5, 1, 0, 2, 0, 1], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 0, 3, 5, 0, 0, 1, 0, 1], [5, 0, 3, 5, 1, 0, 1, 0, 1], [5, 0, 3, 5, 0, 0, 1, 0, 1], [7, 0, 1, 5, 0, 0, 1, 0, 0], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 2, 4, 5, 0, 0, 2, 0, 1], [5, 0, 4, 5, 3, 0, 2, 0, 0], [5, 1, 4, 5, 1, 0, 2, 0, 1], [5, 0, 3, 5, 0, 0, 1, 0, 1], [7, 0, 1, 5, 0, 0, 1, 0, 0], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 2, 4, 5, 0, 0, 2, 0, 1], [5, 0, 4, 5, 3, 0, 2, 0, 0]]
edge_index : [[0, 1, 1, 2, 2, 3, 2, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 14, 16, 16, 17, 17, 18, 18, 19, 18, 20, 20, 21, 21, 22, 21, 23, 23, 24, 24, 25, 24, 5, 24, 8, 20, 9, 18, 12], [1, 0, 2, 1, 3,

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found
